# Data Cleaning + Embedding + FAISS RAG (OpenAI)

## Objective:
Build a complete RAG (Retrieval-Augmented Generation) pipeline using:

- Legal/negotiation/contract documents

- Document cleaning + normalization

- Text chunking

- OpenAI embeddings (text-embedding-3-small)

- FAISS vectorstore (Windows-compatible)

- Semantic search + RAG answer generation

## 1. Imports & configuration

Install all dependencies required for:

- Environment variable loading

- PDF/text reading

- Embedding generation

- FAISS vector storage

- Tracking progress

Key libraries:

- python-dotenv for API key loading

- openai (SDK v1)

- pandas for tracking cleaned files

- tqdm for progress bars

- PyPDF2 for reading PDFs

In [1]:
import os
import json
import glob
import re
import csv
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Any
import dotenv
from dotenv import load_dotenv
from tqdm.auto import tqdm
import hashlib
import pandas as pd
import numpy as np
from PyPDF2 import PdfReader
from openai import OpenAI

try:
    from openai import OpenAI
except Exception:
    OpenAI = None

try:
    from sentence_transformers import SentenceTransformer
except Exception:
    SentenceTransformer = None

c:\Users\user\Documents\ai-legal-negotiation-agent\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Data Loading

**Purpose:**

- Import Python libraries used throughout the notebook

- Load .env

- Initialize OpenAI client

**What happens:**

- API key validation

- Prevents runtime errors early

- Creates a reusable OpenAI client instance

In [2]:
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found in .env file!")

client = OpenAI(api_key=OPENAI_API_KEY)

print("OpenAI API Key loaded successfully.")

OpenAI API Key loaded successfully.


**Purpose:**

Automatically detect your project root without hardcoding paths.
Helps avoid file path errors on different machines.

Folders checked:

- /data

- /backend

- /frontend

**Also sets:**

- Raw documents directory

- Cleaned documents directory

- Vectorstore directory

In [3]:
def find_project_root(start=Path.cwd(), markers=("data", "backend", "frontend")):
    cur = start.resolve()
    for _ in range(10):
        if any((cur / m).exists() for m in markers):
            return cur
        if cur == cur.parent:
            break
        cur = cur.parent
    raise FileNotFoundError("Project root not found.")

try:
    PROJECT_ROOT = find_project_root()
except:
    PROJECT_ROOT = Path(r"C:\Users\user\Documents\ai-legal-negotiation-agent\ai-legal-negotiation-agent")

print("PROJECT_ROOT =", PROJECT_ROOT)

RAW_DIR = PROJECT_ROOT / "data" / "raw" / "generated"
CLEAN_DIR = PROJECT_ROOT / "data" / "clean_openai"
CHROMA_DIR = PROJECT_ROOT / "data" / "vectorstore_openai"

print("RAW_DIR:", RAW_DIR)
print("CLEAN_DIR:", CLEAN_DIR)
print("CHROMA_DIR:", CHROMA_DIR)


PROJECT_ROOT = C:\Users\user\Documents\ai-legal-negotiation-agent\ai-legal-negotiation-agent
RAW_DIR: C:\Users\user\Documents\ai-legal-negotiation-agent\ai-legal-negotiation-agent\data\raw\generated
CLEAN_DIR: C:\Users\user\Documents\ai-legal-negotiation-agent\ai-legal-negotiation-agent\data\clean_openai
CHROMA_DIR: C:\Users\user\Documents\ai-legal-negotiation-agent\ai-legal-negotiation-agent\data\vectorstore_openai


## 3. List Raw files and confirm count

**Purpose:**

Display all raw documents in your project.

Useful for verifying file availability before cleaning.

In [4]:
raw_files = sorted(RAW_DIR.glob("*"))

print(f"📄 Found {len(raw_files)} raw documents.")
for f in raw_files[:10]:
    print(" -", f.name)


📄 Found 56 raw documents.
 - contract_10_20251112_173721.txt
 - contract_11_20251112_173748.txt
 - contract_12_20251112_173817.txt
 - contract_1_20251112_173315.txt
 - contract_2_20251112_173345.txt
 - contract_3_20251112_173421.txt
 - contract_4_20251112_173454.txt
 - contract_5_20251112_173519.txt
 - contract_6_20251112_173542.txt
 - contract_7_20251112_173610.txt


## 4. Cleaning Utilities

**Purpose:**
Defines helper functions for:

- Reading text files

- Extracting PDF text

- Cleaning messy text

- Removing double spaces, extra newlines

- Fingerprinting documents (SHA-256) to identify duplicates

In [5]:
def read_txt(path):
    return path.read_text(encoding="utf-8", errors="ignore")

def read_pdf(path):
    text = ""
    try:
        pdf = PdfReader(str(path))
        for page in pdf.pages:
            text += (page.extract_text() or "") + "\n"
    except:
        pass
    return text

def sanitize(text):
    text = text.replace("\r\n", "\n")
    text = re.sub(r"\n{3,}", "\n\n", text)
    text = re.sub(r"[ \t]+", " ", text)
    return text.strip()

def fingerprint(text):
    return hashlib.sha256(text.encode()).hexdigest()

print("Cleaning helpers ready.")


Cleaning helpers ready.


## 5. Clean, deduplicate, and write cleaned files

**Purpose:**

- Read every raw document

- Convert to clean normalized text

- Skip documents that are too short

- Skip duplicates (via fingerprint hashing)

- Save cleaned version to /data/clean_openai

- Log metadata into a pandas DataFrame

**Outputs:**

- Count of cleaned files

- Count of skipped short files

- Count of skipped duplicates

In [6]:
CLEAN_DIR.mkdir(parents=True, exist_ok=True)

records = []
seen = set()
skipped_short = skipped_dupe = 0

for f in tqdm(raw_files, desc="Cleaning"):
    raw = read_pdf(f) if f.suffix.lower() == ".pdf" else read_txt(f)
    clean = sanitize(raw)

    if len(clean) < 200:
        skipped_short += 1
        continue

    fp = fingerprint(clean)
    if fp in seen:
        skipped_dupe += 1
        continue
    seen.add(fp)

    out = CLEAN_DIR / (f.stem + "_clean.txt")
    out.write_text(clean, encoding="utf-8")

    records.append({"filename": f.name, "clean_file": out.name, "chars": len(clean)})

print("Cleaned documents:", len(records))
print("Skipped short:", skipped_short)
print("Skipped duplicates:", skipped_dupe)

df_clean = pd.DataFrame(records)
df_clean.head()


Cleaning: 100%|██████████| 56/56 [00:00<00:00, 369.55it/s]


Cleaned documents: 56
Skipped short: 0
Skipped duplicates: 0


,filename,clean_file,chars
0,contract_10_20251112_173721.txt,contract_10_20251112_173721_clean.txt,4907
1,contract_11_20251112_173748.txt,contract_11_20251112_173748_clean.txt,5369
2,contract_12_20251112_173817.txt,contract_12_20251112_173817_clean.txt,6359
3,contract_1_20251112_173315.txt,contract_1_20251112_173315_clean.txt,4469
4,contract_2_20251112_173345.txt,contract_2_20251112_173345_clean.txt,4850


## 6. Chunking Documents

**Purpose:**

Split cleaned documents into overlapping chunks (for embeddings).
Chunking improves search recall and avoids embedding very long texts.

Defaults:

- Max length = 1500 characters

- Overlap = 200 characters

**Result:**
A list of chunk dictionaries containing:

- id

- doc (chunk text)

- source (original file name)

In [7]:
def chunk_text(text, max_len=1500, overlap=200):
    chunks = []
    start = 0
    while start < len(text):
        end = start + max_len
        chunks.append(text[start:end].strip())
        start = end - overlap
    return chunks

chunks = []

for i, row in df_clean.iterrows():
    text = (CLEAN_DIR / row["clean_file"]).read_text(encoding="utf-8", errors="ignore")
    
    parts = chunk_text(text)
    for j, c in enumerate(parts):
        chunks.append({
            "id": f"{i+1}_chunk_{j+1}",
            "doc": c,
            "source": row["filename"]
        })

print("Total chunks created:", len(chunks))
chunks[:3]


Total chunks created: 227


[{'id': '1_chunk_1',
  'doc': '**LEASE AGREEMENT**\n\n**THIS LEASE AGREEMENT** (hereinafter referred to as the "Agreement") is made and entered into this ___ day of __________, 20__, by and between:\n\n**LANDLORD:** \nName: ______________________________ \nID Number: _________________________ \nAddress: ____________________________ \nPhone Number: ______________________ \n\n**AND**\n\n**TENANT:** \nName: ______________________________ \nID Number: _________________________ \nAddress: ____________________________ \nPhone Number: ______________________ \n\n**WHEREAS**, the Landlord is the lawful owner of the property located at: \n**Property Address:** ____________________________________________________ \n(hereinafter referred to as the "Premises"); \n\n**AND WHEREAS**, the Tenant wishes to lease the Premises from the Landlord under the terms and conditions set forth herein.\n\n**NOW, THEREFORE, in consideration of the mutual covenants and agreements contained herein, the parties agree 

## 7. Generate OpenAI Embeddings

**Purpose:**

Use OpenAI’s text-embedding-3-small model to convert each text chunk into a numerical vector.

**Features:**

- Batch processing (32 per request)

- Embeddings, IDs, metadata stored in lists

**Result:**

- embeddings → List of embedding vectors

- ids → Chunk IDs

- metadata → Original filenames

In [8]:
def embed(text_batch):
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=text_batch
    )
    return [d.embedding for d in response.data]

embeddings = []
ids = []
metadata = []

BATCH = 32
print("Generating embeddings...")

for i in tqdm(range(0, len(chunks), BATCH), desc="Embedding"):
    batch = chunks[i:i+BATCH]
    texts = [x["doc"] for x in batch]

    embs = embed(texts)

    for c, e in zip(batch, embs):
        embeddings.append(e)
        ids.append(c["id"])
        metadata.append({"source": c["source"]})

print("Embeddings complete:", len(embeddings))


Generating embeddings...


Embedding: 100%|██████████| 8/8 [00:12<00:00,  1.55s/it]

Embeddings complete: 227


## 8. Build FAISS Vectorstore (Windows Compatible)

**Purpose:**

Create and store a FAISS vector index for fast similarity search.

Why FAISS instead of Chroma:

- Chroma 0.5+ breaks old API

- Chroma persistence crashes on Windows

- FAISS works flawlessly and is production-grade

**What this cell does:**

- Creates a FAISS L2 index

- Converts embeddings to float32

- Adds embeddings to FAISS

- Saves index + metadata to disk

**Outputs:**

- documents.index → FAISS binary index

- metadata.pkl → Chunk metadata

In [9]:
import faiss
import numpy as np
import pickle

VECTOR_DIM = len(embeddings[0])

index = faiss.IndexFlatL2(VECTOR_DIM)

emb_matrix = np.array(embeddings).astype('float32')

index.add(emb_matrix)

print("FAISS index size:", index.ntotal)

FAISS_DIR = PROJECT_ROOT / "data" / "vectorstore_faiss"
FAISS_DIR.mkdir(parents=True, exist_ok=True)

faiss.write_index(index, str(FAISS_DIR / "documents.index"))

with open(FAISS_DIR / "metadata.pkl", "wb") as f:
    pickle.dump({
        "ids": ids,
        "chunks": chunks,
        "metadata": metadata
    }, f)

print("FAISS vectorstore saved successfully!")


FAISS index size: 227
FAISS vectorstore saved successfully!


## 9. Load FAISS + Metadata

**Purpose:**

Load FAISS and metadata in any new session.
Allows retrieval without recomputing embeddings.

Loads:

- FAISS index

- Chunk text

- Metadata

- Document IDs

In [10]:
import faiss
import numpy as np
import pickle

FAISS_DIR = PROJECT_ROOT / "data" / "vectorstore_faiss"

index = faiss.read_index(str(FAISS_DIR / "documents.index"))
print("Loaded FAISS index with size:", index.ntotal)

with open(FAISS_DIR / "metadata.pkl", "rb") as f:
    store = pickle.load(f)

ids = store["ids"]
chunks = store["chunks"]
metadata = store["metadata"]

print("Loaded chunks:", len(chunks))


Loaded FAISS index with size: 227
Loaded chunks: 227


## 10. Semantic Search Using OpenAI Embeddings

**Purpose:**
Implement hybrid search:

- Embed the user query

- Search the FAISS index

- Return the most relevant chunks

**Outputs for each match:**

- Chunk text

- Original document name

- L2 distance

In [11]:
def embed_query(text):
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=[text]
    )
    return np.array(response.data[0].embedding).astype("float32")


def search_faiss(query, k=5):
    query_vec = embed_query(query).reshape(1, -1)
    distances, indices = index.search(query_vec, k)
    
    results = []
    for idx, dist in zip(indices[0], distances[0]):
        if idx == -1:
            continue
        results.append({
            "chunk": chunks[idx]["doc"],
            "source": chunks[idx]["source"],
            "distance": float(dist)
        })
    return results

search_faiss("payment terms in the negotiation contract", k=3)


[{'chunk': 'ovided and the terms of payment in accordance with the agreed conditions.)',
  'source': 'contracts_7.txt',
  'distance': 0.8464417457580566},
 {'chunk': "constraints and seeks to negotiate a more favorable rate. The Client presents the following considerations:\n- Budget Cap: KES [Client's Budget Cap]\n- Duration of the Project: [Duration]\n- Expected Outcomes: [Describe expected outcomes]\n\n**5. Negotiation Process**\n\nThe parties agree to the following steps for negotiation:\n- Initial Meeting: A meeting will be held on [Insert Date] at [Insert Time] to discuss the proposed rates.\n- Counter-Proposal: The Client will provide a counter-proposal no later than [Insert Date].\n- Final Agreement: Both parties aim to reach a final agreement by [Insert Date].\n\n**6. Factors Influencing Rate Adjustment**\n\nBoth parties acknowledge that the following factors may influence the final agreed rate:\n- Market Trends: Analysis of current market rates for similar services.\n- Experi

## 11. RAG Answer Generation using OpenAI

**Purpose:**

Build a RAG (Retrieval-Augmented Generation) prompt using the retrieved chunks.

**How it works:**

- Retrieve top-k similar chunks

- Build a context block

- Ask OpenAI to answer ONLY using those chunks

- Return answer + documents used

**Uses:**

- gpt-4.1-mini (fast, low cost, accurate)

In [12]:
def rag_answer(query, k=5):
    results = search_faiss(query, k=k)
    
    context = "\n\n---\n\n".join([r["chunk"] for r in results])

    prompt = f"""
You are an AI legal assistant. Use ONLY the context below to answer the user's question.
If the answer is not in the documents, say: "The documents do not contain that information."

### Context:
{context}

### Question:
{query}

### Answer:
"""

    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=350,
        temperature=0.1,
    )

    answer = response.choices[0].message.content
    return answer, results


## 12. Test the RAG Pipeline

**Purpose:**

Verify that the system retrieves correct documents and generates a grounded, context-based answer.

**Output:**

- Final legal-style answer

- List of chunks used

- Relevance distances

In [13]:
query = "What are the payment terms in the negotiation contract?"
answer, docs = rag_answer(query, k=3)

print(" RAG Answer:\n")
print(answer)

print("\n\n Retrieved Documents Used:\n")
for d in docs:
    print(f"- Source: {d['source']} (distance={d['distance']:.4f})")


 RAG Answer:

The payment terms in the negotiation contract are as follows:

- Payment Schedule: To be determined and inserted (e.g., 50% upfront, 50% upon completion).
- Payment Method: To be specified (e.g., Bank transfer, M-Pesa, etc.).
- Invoicing: The Freelancer will issue invoices upon completion of each milestone.

Additionally, for the supplier-retailer agreement:

- Payment terms are to be detailed in Section 3.1 (not fully provided in the context).
- Any price adjustments must be communicated in writing at least 30 days in advance.

The exact payment schedule and methods are to be agreed upon and inserted in the contract.


 Retrieved Documents Used:

- Source: negotiation_5_20251112_174509.txt (distance=0.8834)
- Source: contracts_7.txt (distance=0.8947)
- Source: negotiation_3_20251112_174423.txt (distance=0.9425)
